In [322]:
from aocd import get_data
data = get_data(day=12, year=2023)

In [308]:
data = '''???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1'''

In [203]:
import re
import itertools
from more_itertools import distinct_permutations

In [233]:
from cProfile import Profile
from pstats import SortKey, Stats

In [325]:
def is_valid(spl, pat):
    #print(pat)
    counter = 0
    pati = 0
    patl = len(pat)
    for c in spl:
        if c == '#':
            counter += 1
        elif counter > 0 and pati < patl:
            if counter != pat[pati]:
                return False
            #print('checked ', counter, '==', pat[pati])
            pati += 1
            counter = 0
    if counter > 0 and pati < patl:
        #print('checked ', counter, '==', pat[pati])
        if counter != pat[pati]:
            return False
        pati += 1
    if pati < len(pat):
        return False
    else:
        return True
    #return [ len(x) for x in ''.join(spl).split('.') if x ] == pat

#is_valid('#.#.###', [1,1,3,4])

def permutation(sp, arr):
    lbs = sp.count('#')
    qs = sp.count('?')
    needed_lbs = sum(arr) - lbs
    needed_dots = qs - needed_lbs
    indexes = [ i for i,c in enumerate(sp) if c == '?' ]
    
    #if needed_lbs + needed_dots > len(indexes):
    #    print('coulda skipped!!')
    #    print(f"    {sp} needs {needed_lbs}# and {needed_dots}. to match {arr}")
    for x in distinct_permutations((['#']*needed_lbs)+['.']*needed_dots):
        spl = list(sp)
        for qi, c in zip(indexes,x):
            spl[qi] = c
        yield spl
        
def smart_permutation(sp, arr):
    lbs = sp.count('#')
    qs = sp.count('?')
    needed_lbs = sum(arr) - lbs
    needed_dots = qs - needed_lbs
    indexes = [ i for i,c in enumerate(sp) if c == '?' ]
    
    #if needed_lbs + needed_dots > len(indexes):
    #    print('coulda skipped!!')
    #    print(f"    {sp} needs {needed_lbs}# and {needed_dots}. to match {arr}")
    for x in distinct_permutations((['#']*needed_lbs)+['.']*needed_dots):
        spl = list(sp)
        for qi, c in zip(indexes,x):
            spl[qi] = c
        yield spl

def doit():
    total = 0
    for i, line in enumerate(data.splitlines()):
        
        sp, arr = line.split(' ')
        arr = [ int(x) for x in arr.split(',') ]
        #print('for', sp, arr)

        for e in permutation(sp, arr):
            if is_valid(e, arr):
                #print('isvalid', e)
                total += 1
    return total
    
with Profile() as profile:
    print(doit())
    (
        Stats(profile).sort_stats(SortKey.CALLS).print_stats()
    )
    


8270
         3380021 function calls in 2.471 seconds

   Ordered by: call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   849202    0.028    0.000    0.028    0.000 {built-in method builtins.len}
   840929    1.066    0.000    1.822    0.000 /var/folders/5t/xztjvk6j5v98pymbd691_4080000gp/T/ipykernel_14128/3611103315.py:28(permutation)
   840929    0.754    0.000    0.754    0.000 /Users/matthew.gill/.pyenv/versions/3.11.3/lib/python3.11/site-packages/more_itertools/more.py:664(_full)
   839929    0.435    0.000    0.463    0.000 /var/folders/5t/xztjvk6j5v98pymbd691_4080000gp/T/ipykernel_14128/3611103315.py:1(is_valid)
     2000    0.000    0.000    0.000    0.000 {method 'split' of 'str' objects}
     2000    0.000    0.000    0.000    0.000 {method 'count' of 'str' objects}
     1000    0.000    0.000    0.000    0.000 {built-in method builtins.sorted}
     1000    0.000    0.000    0.000    0.000 {built-in method builtins.sum}
     1000    0.001    

In [370]:
from functools import cache

# this one goes to the internet, i was interested. very very fast.
def total_combinations(springs, groups):
    @cache
    def combinations(i, j, r):
        if j == len(groups):
            return springs.count("#", i) == 0
        if i == len(springs):
            return j == len(groups) - 1 and r == groups[j]
        n = 0
        if springs[i] != ".":
            n += combinations(i + 1, j, r + 1)
        if springs[i] != "#":
            if r == groups[j]:
                n += combinations(i + 1, j + 1, 0)
            elif r == 0:
                n += combinations(i + 1, j, 0)
        return n

    return combinations(0, 0, 0)

# thanks chatgpt
@cache
def count_arrangements(row, groups, done=0):
    if not row:
        return not groups and not done

    total_arrangements = 0
    row = row if row.endswith(".") else row + "."
    
    if row.startswith("?"):
        cells = ".#"
    else:
        cells = row[:1]
    
    for cell in cells:
        #print(current_group)
        if cell == '#':
        #if row.count('?', i) == current_group:
            total_arrangements += count_arrangements(row[1:], groups, done+1)
        elif done and groups and groups[0] == done:
            total_arrangements += count_arrangements(row[1:], groups[1:])
        elif not done:
            total_arrangements += count_arrangements(row[1:], groups)
    
    return total_arrangements

total = 0

for i, line in enumerate(data.splitlines()):
    sp, arr = line.split(' ')
    arr = tuple([ int(x) for x in arr.split(',') ])
    
    sp = '?'.join([sp]*5)
    arr = arr * 5
    #total += total_combinations(sp, arr)
    total += count_arrangements(sp, arr)
total

204640299929836